In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from django.db.models import Q

In [3]:
season = Season.objects.all().last()
season

<Season: 3: 2019 Fall Start:2019-09-16 End:2020-01-05>

In [4]:
meeting = Meeting.objects.filter(season=season).get(date='2019-12-06')

In [5]:
scheduled = Schedule.objects.filter(meeting=meeting)
guy_ids = scheduled.filter(player__gender='M').values_list('player__id', 'partner_id')
guy_ids_only = scheduled.filter(player__gender='M').values_list('player__id', flat=True)
gal_ids = scheduled.filter(Q(player__gender='F')&~Q(partner__in=guy_ids_only)).values_list('partner_id', 'player__id')

guy_ids, gal_ids

(<QuerySet [(54, None), (40, None), (17, 18), (1, 2), (113, 112), (135, 134), (19, 111), (61, 66)]>,
 <QuerySet []>)

In [6]:
couples = set(guy_ids).union(gal_ids)
list(couples)

[(17, 18),
 (1, 2),
 (135, 134),
 (54, None),
 (19, 111),
 (61, 66),
 (113, 112),
 (40, None)]

In [10]:
import json
from TBLib.serializers import PlayerSerializer
guys = Player.objects.filter(id__in=scheduled.filter(player__gender='M').values('player__id'))
sguys = PlayerSerializer(guys, many=True)
json.dumps(sguys.data)

'[{"id": 40, "full_name": "Wes Carr", "ntrp": 3.5, "microntrp": 3.8, "gender": "M"}, {"id": 54, "full_name": "Fazal Shariff", "ntrp": 4.0, "microntrp": 4.0, "gender": "M"}, {"id": 19, "full_name": "Jeff Trask", "ntrp": 4.5, "microntrp": 4.5, "gender": "M"}, {"id": 17, "full_name": "Mark Smith", "ntrp": 4.5, "microntrp": 4.5, "gender": "M"}, {"id": 61, "full_name": "Willie Lafrades", "ntrp": 4.0, "microntrp": 4.1, "gender": "M"}, {"id": 1, "full_name": "Ed Henderson", "ntrp": 3.5, "microntrp": 3.85, "gender": "M"}, {"id": 113, "full_name": "Jason Steele", "ntrp": 3.5, "microntrp": 3.9, "gender": "M"}, {"id": 135, "full_name": "Jeff Thompson", "ntrp": 4.0, "microntrp": 4.15, "gender": "M"}]'

In [ ]:
sguy